# Load library

In [1]:
from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

# SentenceTokenizer Class

In [2]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
        ,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",'로써',
        '해당', '니다']
    def url2sentences(self, url):                   # URL일 경우 Text 추출
        article = Article(url, language='ko')
        article.download()
        article.parse()
        sentences = self.kkma.sentences(article.text)   # 텍스트에서 문장별로 반환
        print(article.text)
        print('*'*120)

        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
            return sentences

    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences

    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
              nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence))
            if noun not in self.stopwords and len(noun) > 1]))
        return nouns


# GrapMatrix Class

In [3]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence

    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

# Rank Class

In [4]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1

        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

# TextRank Class

In [5]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        if text[:5] in ('http:', 'https'):
            self.sentences = self.sent_tokenize.url2sentences(text)
        else:
            self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)

    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)

        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])

        return summary

    def keywords(self, word_num=15):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)

        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)

        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords


# Main

In [6]:
url = input()
textrank = TextRank(url)
print(textrank.keywords())
for row in textrank.summarize(5):
    print(row)
    print()
print('keywords :',textrank.keywords())

안녕하세요 쏘선생님입니다.

대학교를 목표로 하시는군요, 애견학과가 있는 학교로는

대표적으로 원광대, 신구대, 연희대 등이있습니다.

애견과인만큼, 이론이나 실무를 공부하겠죠.

만약 취업을 목적으로 하시는거라면, 저는 대학교는 추천드리지 않습니다.

애견분야는 학력이 아닌, 실력과 자격증으로 손님들이 오시는거고,

해당 취업기관도 실무능력을 더 우선시합니다.

애견학과에 나왔다고 우대를 하는게 아닌, 실력중심으로 우대를 하기에,

실력이 있어야 취득할 수 있는 자격증을 더 봅니다.

애견분야도 여러 분야가 있습니다. 정확히 어느 분야로 취업을 원하시나요?

만약 취업을 원하시는거라면, 저는 자격증을 추천드립니다.

저는 반려동물관리사 선생님입니다.

애견과 관련된 자격증을 취득하고싶거나, 더 자세히 알고싶으시다면

질문해주세요. 정확하게 답변해드리겠습니다.

자신이 원하는 분야와, 목표를 세우고 자격증을 준비하세요.

유능한 선생님과 상담해보세요.

문의주시면 반려동물관리사 직업 자료와 핵심기출문제 보내드립니다^^

(상담문의)

http://naver.me/I5jden8H

핸드폰으로 문자나 카톡 남겨주셔도 됩니다
************************************************************************************************************************
['자격증', '애견', '취업', '실무', '능력', '손님', '우선', '기관', '시합', '학력', '오시', '문의', '실력', '문제', '자료']
만약 취업을 목적으로 하시는 거라면, 저는 대학교는 추천 드리지 않습니다.

우대를 하는 게 아닌, 실력 중심으로 우대를 하기에, 실력이 있어야 취득할 수 있는 자격증을 더 봅니다.

애견분야도 여러 분야가 있습니다.

정확히 어느 분야로 취업을 원하시나요?

만약 취업을 원하시는 거라면, 저는 자격증을 추천 드립니다.

keywords : ['자격증', '애견', '취

In [11]:
raw = input()

In [13]:
textrank.keywords()

['자격증',
 '애견',
 '취업',
 '실무',
 '능력',
 '손님',
 '우선',
 '기관',
 '시합',
 '오시',
 '학력',
 '문의',
 '실력',
 '문제',
 '자료']

In [12]:
for i in textrank.keywords():
    print(i + ' : ' + str(raw.count(i)))

자격증 : 5
애견 : 6
취업 : 4
실무 : 2
능력 : 1
손님 : 1
우선 : 1
기관 : 1
시합 : 1
학력 : 1
오시 : 1
문의 : 2
실력 : 3
문제 : 1
자료 : 1
